In [ ]:
# default_exp trainer.talknet

In [ ]:
# export
import os
import pytorch_lightning as pl
from nemo.collections.tts.models import (
    TalkNetPitchModel,
    TalkNetDursModel,
    TalkNetSpectModel,
)
from nemo.utils.exp_manager import exp_manager
from nemo.collections.common.callbacks import LogEpochTimeCallback


def train_spect(cfg):

    output_dir = cfg.output_dir
    batch_size = cfg.batch_size
    learning_rate = cfg.learning_rate
    min_learning_rate = cfg.min_learning_rate
    epochs = cfg.epochs
    cfg.sample_rate = 22050
    cfg.train_dataset = cfg.trainfiles_json_path
    # NOTE (Sam): can fix later
    cfg.validation_datasets = cfg.trainfiles_json_path
    cfg.durs_file = os.path.join(output_dir, "durations.pt")
    cfg.f0_file = os.path.join(output_dir, "f0s.pt")
    cfg.trainer.accelerator = "dp"
    cfg.trainer.max_epochs = epochs
    cfg.trainer.check_val_every_n_epoch = 5
    cfg.model.train_ds.dataloader_params.batch_size = batch_size
    cfg.model.validation_ds.dataloader_params.batch_size = batch_size
    cfg.model.optim.lr = learning_rate
    cfg.model.optim.sched.min_lr = min_learning_rate
    cfg.exp_manager.exp_dir = output_dir
    
    warmstart_path = cfg.warmstart_path
    trainer = pl.Trainer(**cfg.trainer)
    model = TalkNetSpectModel.restore_from(warmstart_path, override_config_path=cfg)
    model.set_trainer(trainer)
    model.setup_training_data(cfg.model.train_ds)
    model.setup_validation_data(cfg.model.validation_ds)
    model.setup_optimization(cfg.model.optim)
    print("Warm-starting from " + warmstart_path)
    exp_manager(trainer, cfg.get("exp_manager", None))
    trainer.callbacks.extend(
        [pl.callbacks.LearningRateMonitor(), LogEpochTimeCallback()]
    )  # noqa
    trainer.fit(model)


def train_durs(cfg):

    cfg.sample_rate = 22050
    cfg.train_dataset = cfg.trainfiles_json_path
    cfg.validation_datasets = cfg.trainfiles_json_path
    output_dir = cfg.output_dir
    warmstart_path = cfg.warmstart_path
    cfg.durs_file = os.path.join(output_dir, "durations.pt")
    cfg.f0_file = os.path.join(output_dir, "f0s.pt")
    cfg.trainer.accelerator = "dp"
    cfg.trainer.max_epochs = 10
    cfg.trainer.check_val_every_n_epoch = 5
    cfg.model.train_ds.dataloader_params.batch_size = 16
    cfg.model.validation_ds.dataloader_params.batch_size = 16
    cfg.model.optim.lr = 0.001
    cfg.model.optim.sched.min_lr = 1e-5
    cfg.exp_manager.exp_dir = output_dir

    # Find checkpoints
    # warmstart_path = "/content/talknet_durs.nemo"
    trainer = pl.Trainer(**cfg.trainer)
    model = TalkNetDursModel.restore_from(warmstart_path, override_config_path=cfg)
    model.set_trainer(trainer)
    model.setup_training_data(cfg.model.train_ds)
    model.setup_validation_data(cfg.model.validation_ds)
    model.setup_optimization(cfg.model.optim)
    print("Warm-starting duration model from " + warmstart_path)
    exp_manager(trainer, cfg.get("exp_manager", None))
    trainer.callbacks.extend(
        [pl.callbacks.LearningRateMonitor(), LogEpochTimeCallback()]
    )  # noqa
    trainer.fit(model)


def train_pitch(cfg):
    batch_size = cfg.batch_size
    learning_rate = cfg.learning_rate
    min_learning_rate = cfg.min_learning_rate
    output_dir = cfg.output_dir

    cfg.sample_rate = 22050
    cfg.train_dataset = cfg.trainfiles_json_path
    cfg.validation_datasets = cfg.trainfiles_json_path
    cfg.durs_file = os.path.join(output_dir, "durations.pt")
    cfg.f0_file = os.path.join(output_dir, "f0s.pt")
    cfg.trainer.accelerator = "dp"
    # cfg.trainer.max_epochs = epochs
    cfg.trainer.check_val_every_n_epoch = 5
    # cfg.model.f0_mean = f0_mean
    # cfg.model.f0_std = f0_std
    cfg.model.train_ds.dataloader_params.batch_size = batch_size
    cfg.model.validation_ds.dataloader_params.batch_size = batch_size
    cfg.model.optim.lr = learning_rate
    cfg.model.optim.sched.min_lr = min_learning_rate
    cfg.exp_manager.exp_dir = output_dir

    warmstart_path = cfg.warmstart_path
    trainer = pl.Trainer(**cfg.trainer)
    model = TalkNetPitchModel.restore_from(warmstart_path, override_config_path=cfg)
    model.set_trainer(trainer)
    model.setup_training_data(cfg.model.train_ds)
    model.setup_validation_data(cfg.model.validation_ds)
    model.setup_optimization(cfg.model.optim)
    print("Warm-starting from " + warmstart_path)
    exp_manager(trainer, cfg.get("exp_manager", None))
    trainer.callbacks.extend(
        [pl.callbacks.LearningRateMonitor(), LogEpochTimeCallback()]
    )  # noqa
    trainer.fit(model)